In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import numpy as np
import fireducks.pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
import torch.utils.data as data

from esm.utils.structure.protein_chain import ProteinChain
from esm.models.esm3 import ESM3
from esm.models.esmc import ESMC

from esm.utils.constants.esm3 import SEQUENCE_VOCAB
from esm.sdk.api import (
    ESMProtein,
    ESMProteinTensor,
    GenerationConfig,
    LogitsConfig,
)
from esm.utils.decoding import decode_structure
from esm.utils.constants.esm3 import STRUCTURE_BOS_TOKEN, STRUCTURE_EOS_TOKEN
from esm.utils.residue_constants import atom_order
from esm.layers.rotary import RotaryEmbedding
from esm.layers.blocks import swiglu_ln_ffn, gelu_ln_ffn


In [4]:
seq_7bny = "SPNPLDVSKTYPTLHILLQFNHRGLEARIFRHGQLWAETHAEVVLRSKTKQISFLSNGSYPSMDATTPLNPWKSTYQAVLRAEPHRVTMDVYHKRIRPFRLPLVQKEWRTCEENVFGLYHVFETHYAGYFSDLLIHDVETNPGGSKHHHHHH"
protein = ESMProtein(sequence=seq_7bny)
model = ESMC.from_pretrained("esmc_300m", device=torch.device("cpu"))
protein_tensor = model.encode(protein).to(torch.device("cpu"))
logits_output = model.logits(
   protein_tensor, LogitsConfig(sequence=True, structure=True, secondary_structure=True, 
   sasa=True, function=True, return_embeddings=True, return_hidden_states=True)
)
print(logits_output.logits, logits_output.embeddings)

ValueError: Expected a torch.device with a specified index or an integer, but got:None